In [1]:
%matplotlib inline
%pip install statsmodels

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import os
from scipy.stats import zscore
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
from scipy.stats import kruskal

import warnings

warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [7]:
from pro1 import df_order, df_sellers, df_order_items, df_join_order_cp, df_order_reviews, df_join_ocpi, df_products, df_product_category_name_translation

# from pro1 import df_customers, df_geolocation, df_order_items, df_order_payments, df_order_reviews, df_products, df_sellers

과제 3: 상품 카테고리별 수요 예측 및 재고 관리 인사이트
시간에 따른 상품 카테고리별 판매 트렌드를 분석하고, 계절성 패턴을 파악하여 향후 수요 예측과 재고 관리 전략을 수립하세요

In [9]:
print(df_products.columns)
print(df_product_category_name_translation.columns)

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')
Index(['product_category_name', 'product_category_name_english'], dtype='object')


In [10]:
print(df_products.columns)
print(df_product_category_name_translation.columns)

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')
Index(['product_category_name', 'product_category_name_english'], dtype='object')


In [11]:
# 혹시 컬럼명에 공백/따옴표가 있을 수 있으니 정리
df_product_category_name_translation.rename(columns={'ï»¿product_category_name': 'product_category_name'}, inplace=True)

merge_product_cate = df_products.merge(
    df_product_category_name_translation,
    on="product_category_name",
    how="left"
)

# 확인
# print(merge_product_cate.head())
# merge_product_cate.isnull().sum()

'''merge_product_cate에서 결측치가 발생한 이유는 두 가지예요:

제품 데이터 자체에 결측치가 있음 → 예: product_weight_g, product_length_cm 등 (2건)

카테고리명이 번역 테이블에 없음 → product_category_name_english에 623개 NaN'''

# merge 후 결측치 처리
# 수치형은 NaN → 중앙값으로 대체
# 카테고리 번역 없는 경우 → 원래 포르투갈어 명칭 유지 (unknown까지 커버 가능)

# 1. 수치형 컬럼 결측치 → 중앙값으로 채우기
num_cols = ["product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm"]
for col in num_cols:
    median_val = merge_product_cate[col].median()
    merge_product_cate[col].fillna(median_val, inplace=True)

# 2. 카테고리 번역 결측치 처리
#   - 번역이 없는 경우 product_category_name 그대로 사용
merge_product_cate["product_category_name_english"] = merge_product_cate["product_category_name_english"].fillna(
    merge_product_cate["product_category_name"].fillna("unknown")
)

# 결측치 삭제 : 분석 목표가 “제품별 성능/특징을 기반으로 한 분석”이라면, 정보가 없는 제품은 분석 불가능 → 삭제하는 게 깔끔합니다.
merge_product_cate = merge_product_cate.dropna(
    subset=["product_category_name", "product_name_lenght", 
            "product_description_lenght", "product_photos_qty"]
)

# 확인
print(merge_product_cate.isnull().sum())

product_id                       0
product_category_name            0
product_name_lenght              0
product_description_lenght       0
product_photos_qty               0
product_weight_g                 0
product_length_cm                0
product_height_cm                0
product_width_cm                 0
product_category_name_english    0
dtype: int64


C:\Users\mumu1\AppData\Local\Temp\ipykernel_5564\2882898953.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge_product_cate[col].fillna(median_val, inplace=True)
C:\Users\mumu1\AppData\Local\Temp\ipykernel_5564\2882898953.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [ ]:
# merge_product_cate + df_join_ocpi 병합

merge_full = merge_product_cate.merge(
    df_join_ocpi,
    on="product_id",
    how="inner"   # 보통 inner join, 필요하면 left join으로 변경 가능
)

# 확인
# print(merge_full.isnull().sum())

merge_full.info()

In [ ]:
# 분석 절차
# 주문 시점(order_purchase_timestamp)을 월 단위로 변환
# product_category_name_english 기준으로 판매량 집계
# 판매량 기준: order_item_id 건수 또는 price 합계 (둘 다 가능)
# 시계열 트렌드 DataFrame 생성

# 주문 날짜 컬럼을 datetime으로 변환
merge_full["approved_to_carrier"] = pd.to_datetime(merge_full["approved_to_carrier"])
merge_full["approved_to_carrier"] = pd.to_datetime(merge_full["approved_to_carrier"])

# 월 단위 컬럼 추가
merge_full["order_month"] = merge_full["order_purchase_timestamp"].dt.to_period("M").dt.to_timestamp()

# 1. 카테고리별 판매량(=주문 건수)
category_trend_qty = (
    merge_full
    .groupby(["order_month", "product_category_name_english"])
    .size()
    .reset_index(name="sales_qty")
)

# 2. 카테고리별 매출액(=price 합계)
category_trend_sales = (
    merge_full
    .groupby(["order_month", "product_category_name_english"])["price"]
    .sum()
    .reset_index(name="sales_value")
)

# 확인
print(category_trend_qty.head())
print(category_trend_sales.head())


In [ ]:
# 시각화 스타일 설정
sns.set_style("whitegrid")
plt.rc('font', size=10) # 폰트 크기 설정

## 1. 판매량 시각화
plt.figure(figsize=(15, 8))
sns.lineplot(data=category_trend_qty, x="order_month", y="sales_qty", hue="product_category_name_english")
plt.title("월별 상품 카테고리 판매량(주문 건수) 트렌드", fontsize=15, pad=20)
plt.xlabel("주문 월", fontsize=12)
plt.ylabel("판매량(주문 건수)", fontsize=12)
plt.legend(title="카테고리", bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

## 2. 매출액 시각화
plt.figure(figsize=(15, 8))
sns.lineplot(data=category_trend_sales, x="order_month", y="sales_value", hue="product_category_name_english")
plt.title("월별 상품 카테고리 매출액 트렌드", fontsize=15, pad=20)
plt.xlabel("주문 월", fontsize=12)
plt.ylabel("매출액", fontsize=12)
plt.legend(title="카테고리", bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

# 바 그래프 시각화
# 카테고리별 총 매출액 계산 (category_trend_sales 데이터프레임 사용)
# `product_category_name_english`별로 `sales_value`의 합계를 계산
category_total_sales = category_trend_sales.groupby("product_category_name_english")["sales_value"].sum().reset_index()

# 매출액이 높은 순서로 정렬
category_total_sales = category_total_sales.sort_values(by="sales_value", ascending=False)

# 바 그래프 시각화
plt.figure(figsize=(15, 10))
sns.barplot(
    data=category_total_sales.head(15), # 상위 15개 카테고리만 시각화
    x="sales_value",
    y="product_category_name_english",
    palette="viridis" # 색상 팔레트 지정
)
plt.title("상품 카테고리별 총 매출액", fontsize=15, pad=20)
plt.xlabel("총 매출액", fontsize=12)
plt.ylabel("상품 카테고리", fontsize=12)
plt.show()


In [ ]:
# 특정 인기 카테고리 몇 개만 선택 (예: 상위 5개)
top_categories = (
    category_trend_qty.groupby("product_category_name_english")["sales_qty"]
    .sum()
    .nlargest(5)
    .index
)

plt.figure(figsize=(12,6))

for cat in top_categories:
    subset = category_trend_qty[category_trend_qty["product_category_name_english"] == cat]
    plt.plot(subset["order_month"], subset["sales_qty"], marker="o", label=cat)

plt.title("월별 상품 카테고리별 판매량 트렌드 (Top 5)")
plt.xlabel("주문월")
plt.ylabel("판매량")
plt.legend()
plt.grid(True)
plt.show()

In [12]:
merge_full.head()

NameError: name 'merge_full' is not defined

In [ ]:
'''월·카테고리 집계 생성(판매량/매출),
계절 지표 산출(월별 시즌 인덱스, STL 기반 계절성 강도, ACF@12, 비모수 검정),
상위 카테고리 시계열·ACF·STL 구성요소·월-오브-이어 히트맵 시각화,
강한 계절 카테고리 자동 리포트.'''

# ===== 기본 점검 =====
required_cols = {
    'product_category_name_english',
    'order_month',               # datetime64[ns], 월 첫째날이면 베스트
    'price',                     # 매출 계산용
    'order_item_id'              # Olist 구조에서 '수량' 대용 (행==아이템)
}
missing = required_cols - set(merge_full.columns)
assert len(missing)==0, f"다음 컬럼이 필요합니다: {missing}"

# ===== 1) 월·카테고리 집계 =====
def make_monthly(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['order_month'] = pd.to_datetime(out['order_month']).dt.to_period('M').dt.start_time
    g = out.groupby(['order_month','product_category_name_english'], as_index=False)
    monthly = g.agg(items=('order_item_id','count'),
                    revenue=('price','sum'))
    return monthly.sort_values(['product_category_name_english','order_month'])

monthly_cat = make_monthly(merge_full)

date_min = monthly_cat['order_month'].min()
date_max = monthly_cat['order_month'].max()
print(f"[기간] {date_min.date()} ~ {date_max.date()}")
print(monthly_cat.head())


In [ ]:
# 2) 계절성 측정 도구(시즌 인덱스, STL 강도, ACF@12, Kruskal 검정)
# 카테고리별 시계열 시리즈 생성(빈 달은 0으로 채움)
def series_by_category(monthly, category, metric='items'):
    idx = pd.date_range(monthly['order_month'].min(),
                        monthly['order_month'].max(), freq='MS')
    s = (monthly[monthly['product_category_name_english']==category]
         .set_index('order_month')[metric]
         .reindex(idx, fill_value=0))
    s.index.name = 'order_month'
    return s

# 월-오브-이어(1~12월) 시즌 인덱스: 해당 월 평균 / 전체 평균
def seasonal_index_moy(s: pd.Series) -> pd.Series:
    df = s.to_frame('y')
    df['month'] = df.index.month
    moy = df.groupby('month')['y'].mean()
    overall = df['y'].mean()
    idx = (moy / overall).rename('seasonal_index')
    return idx  # 1.0=평균, 1.2=해당 월이 평균 대비 +20%

# STL 기반 계절성 강도(Hyndman): 1 - Var(resid)/Var(seasonal+resid)
def seasonal_strength_via_stl(s: pd.Series, period=12):
    res = STL(s, period=period, robust=True).fit()
    resid_var = np.var(res.resid, ddof=1)
    season_resid_var = np.var(res.seasonal + res.resid, ddof=1)
    strength = max(0.0, 1.0 - (resid_var / season_resid_var)) if season_resid_var>0 else 0.0
    return float(strength), res

# 월(1~12) 간 분포 차이 비모수 검정(계절성 존재 여부의 보조 지표)
def month_kruskal(s: pd.Series):
    df = s.to_frame('y')
    df['month'] = df.index.month
    groups = [g['y'].values for _, g in df.groupby('month')]
    if len(groups)<2: 
        return np.nan, np.nan
    stat, p = kruskal(*groups)
    return float(stat), float(p)

# 카테고리 요약 리포트 생성
def build_seasonality_report(monthly, categories, metric='items'):
    rows = []
    for cat in categories:
        s = series_by_category(monthly, cat, metric)
        if len(s) < 18:  # 최소 18개월 권장
            continue
        strength, stl_res = seasonal_strength_via_stl(s, period=12)
        idx = seasonal_index_moy(s)
        peak_month = int(idx.idxmax())   # 1~12
        trough_month = int(idx.idxmin())
        peak_trough_ratio = float(idx.max()/idx.min()) if idx.min()>0 else np.nan
        acf_vals = acf(s, nlags=24, fft=False)
        acf12 = float(acf_vals[12]) if len(acf_vals) > 12 else np.nan
        stat, p = month_kruskal(s)

        rows.append({
            'category': cat,
            'metric': metric,
            'n_months': int(len(s)),
            'seasonal_strength_STL': strength,
            'acf_lag12': acf12,
            'kruskal_p': p,
            'peak_month': peak_month,
            'trough_month': trough_month,
            'peak/trough_ratio': peak_trough_ratio
        })
    rep = pd.DataFrame(rows)
    if not rep.empty:
        rep = rep.sort_values(['seasonal_strength_STL','acf_lag12'], ascending=False)
    return rep
